In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pylab as plt
#%pip install seaborn
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator


from sklearn import set_config

#%pip install dagshub mlflow optuna
import mlflow
import mlflow.sklearn
import dagshub


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

import optuna


set_config(transform_output="pandas")
plt.style.use('fivethirtyeight')

In [5]:
dagshub.init(repo_owner='Arej02', repo_name='swiggy_delivery_time_prediction', mlflow=True)

Initialized MLflow to track repo "Arej02/swiggy_delivery_time_prediction"

Repository Arej02/swiggy_delivery_time_prediction initialized!

In [6]:
swiggy=pd.read_csv("swiggy_cleaned.csv")

In [7]:
swiggy.sample(3)

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,time_taken,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance
31718,LUDHRES13DEL03,36.0,4.9,30.893234,75.821720,30.923234,75.851720,2022-02-11,fog,low,...,16,LUDH,11,2,friday,0,10.0,23.0,night,4.395413
32679,CHENRES16DEL02,39.0,5.0,13.049645,80.242268,13.099645,80.292268,2022-03-30,fog,low,...,19,CHEN,30,3,wednesday,0,15.0,22.0,night,7.761422
35830,JAPRES14DEL01,34.0,4.8,26.888420,75.800689,26.958420,75.870689,2022-03-25,sandstorms,low,...,25,JAP,25,3,friday,0,15.0,23.0,night,10.428262


In [8]:
swiggy['distance'].describe()

,distance
count,41872.000000
mean,9.719296
std,5.602890
min,1.465067
25%,4.657655
50%,9.193014
75%,13.680920
max,20.969489


In [9]:
def bin_distance(df:pd.DataFrame,num_col):
    distance_labels=['short','medium','long','very long']
    distance_bins=[1,6,12,18,24]
    df['distance_bins']=pd.cut(df[num_col],bins=distance_bins,labels=distance_labels,right=False)
    return df

In [10]:
bin_distance(swiggy,'distance')

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,INDO,19,3,saturday,1,15.0,11.0,morning,3.025149,short
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,BANG,25,3,friday,0,5.0,19.0,evening,20.183530,very long
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,BANG,19,3,saturday,1,15.0,8.0,morning,1.552758,short
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,COIMB,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,CHEN,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,JAP,24,3,thursday,0,10.0,11.0,morning,1.489846,short
45498,AGRRES16DEL01,21.0,4.6,NaN,NaN,NaN,NaN,2022-02-16,windy,jam,...,AGR,16,2,wednesday,0,15.0,19.0,evening,NaN,NaN
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,CHEN,11,3,friday,0,15.0,23.0,night,4.657195,short
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,COIMB,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium


In [11]:
swiggy['order_time_hour'].describe()

,order_time_hour
count,43862.000000
mean,17.423966
std,4.817856
min,0.000000
25%,15.000000
50%,19.000000
75%,21.000000
max,23.000000


In [12]:
def bin_time(df:pd.DataFrame,num_col):
    time_labels=['after_midnight','morning','afternoon','evening','night']
    time_bins=[0,6,12,17,20,24]
    df['order_time_bins']=pd.cut(df[num_col],bins=time_bins,labels=time_labels,right=False)
    return df

In [13]:
bin_time(swiggy,'order_time_hour')

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins,order_time_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,19,3,saturday,1,15.0,11.0,morning,3.025149,short,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,25,3,friday,0,5.0,19.0,evening,20.183530,very long,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,19,3,saturday,1,15.0,8.0,morning,1.552758,short,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,24,3,thursday,0,10.0,11.0,morning,1.489846,short,morning
45498,AGRRES16DEL01,21.0,4.6,NaN,NaN,NaN,NaN,2022-02-16,windy,jam,...,16,2,wednesday,0,15.0,19.0,evening,NaN,NaN,evening
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,11,3,friday,0,15.0,23.0,night,4.657195,short,night
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium,afternoon


In [14]:
df1=swiggy.copy().dropna()
df1

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins,order_time_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,19,3,saturday,1,15.0,11.0,morning,3.025149,short,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,25,3,friday,0,5.0,19.0,evening,20.183530,very long,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,19,3,saturday,1,15.0,8.0,morning,1.552758,short,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45496,RANCHIRES16DEL01,35.0,4.2,23.371292,85.327872,23.481292,85.437872,2022-03-08,windy,jam,...,8,3,tuesday,0,10.0,21.0,night,16.600272,long,night
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,24,3,thursday,0,10.0,11.0,morning,1.489846,short,morning
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,11,3,friday,0,15.0,23.0,night,4.657195,short,night
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium,afternoon


In [15]:
columns_to_drop=[
    'rider_id',
    'restaurant_latitude',
    'restaurant_longitude',
    'delivery_latitude',
    'delivery_longitude',
    'order_date',
    'order_time_of_day',
    "order_time_hour",
    "order_day",
    "city_name",
    "order_day_of_week",
    "order_month"
]

In [16]:
df1.drop(columns=columns_to_drop,inplace=True)

In [17]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38064 entries, 0 to 45501
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   age                  38064 non-null  float64 
 1   ratings              38064 non-null  float64 
 2   weather              38064 non-null  object  
 3   traffic              38064 non-null  object  
 4   vehicle_condition    38064 non-null  int64   
 5   type_of_order        38064 non-null  object  
 6   type_of_vehicle      38064 non-null  object  
 7   multiple_deliveries  38064 non-null  float64 
 8   festival             38064 non-null  object  
 9   city_type            38064 non-null  object  
 10  time_taken           38064 non-null  int64   
 11  is_weekend           38064 non-null  int64   
 12  pickup_time_minutes  38064 non-null  float64 
 13  distance             38064 non-null  float64 
 14  distance_bins        38064 non-null  category
 15  order_time_bins      380

In [18]:
df1.isnull().mean()

,0
age,0.0
ratings,0.0
weather,0.0
traffic,0.0
vehicle_condition,0.0
type_of_order,0.0
type_of_vehicle,0.0
multiple_deliveries,0.0
festival,0.0
city_type,0.0


In [19]:
df1.duplicated().sum()

np.int64(0)

In [20]:
X=df1.drop(columns=['time_taken'])
X

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,is_weekend,pickup_time_minutes,distance,distance_bins,order_time_bins
0,37.0,4.9,sunny,high,2,snack,motorcycle,0.0,no,urban,1,15.0,3.025149,short,morning
1,34.0,4.5,stormy,jam,2,snack,scooter,1.0,no,metropolitian,0,5.0,20.183530,very long,evening
2,23.0,4.4,sandstorms,low,0,drinks,motorcycle,1.0,no,urban,1,15.0,1.552758,short,morning
3,38.0,4.7,sunny,medium,0,buffet,motorcycle,1.0,no,metropolitian,0,10.0,7.790401,medium,evening
4,32.0,4.6,cloudy,high,1,snack,scooter,1.0,no,metropolitian,1,15.0,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45496,35.0,4.2,windy,jam,2,drinks,motorcycle,1.0,no,metropolitian,0,10.0,16.600272,long,night
45497,30.0,4.8,windy,high,1,meal,motorcycle,0.0,no,metropolitian,0,10.0,1.489846,short,morning
45499,30.0,4.9,cloudy,low,1,drinks,scooter,0.0,no,metropolitian,0,15.0,4.657195,short,night
45500,20.0,4.7,cloudy,high,0,snack,motorcycle,1.0,no,metropolitian,0,5.0,6.232393,medium,afternoon


In [21]:
Y=df1['time_taken']
Y

,time_taken
0,24
1,33
2,26
3,21
4,30
...,...
45496,33
45497,32
45499,16
45500,26


In [22]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=10)

In [23]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (30451, 15)
The shape of test data is (7613, 15)


In [24]:
numerical_col=[cols for cols in df1.columns if df1[cols].dtype in ['int64','float64'] and cols !='time_taken']
numerical_col

['age',
 'ratings',
 'vehicle_condition',
 'multiple_deliveries',
 'is_weekend',
 'pickup_time_minutes',
 'distance']

In [25]:
categorical_col=[cols for cols in df1.columns if df1[cols].dtype in ['object','category']]
categorical_col

['weather',
 'traffic',
 'type_of_order',
 'type_of_vehicle',
 'festival',
 'city_type',
 'distance_bins',
 'order_time_bins']

In [26]:
nominal_col=['weather','type_of_order','type_of_vehicle','festival','distance_bins','order_time_bins']
ordinal_col=['traffic','city_type']

In [27]:
len(numerical_col+nominal_col+ordinal_col)

15

In [28]:
df1['traffic'].unique()

array(['high', 'jam', 'low', 'medium'], dtype=object)

In [29]:
df1['city_type'].unique()

array(['urban', 'metropolitian', 'semi-urban'], dtype=object)

In [30]:
traffic_groups=['low','medium','high','jam']
city_groups=['semi-urban','urban','metropolitian']

In [31]:
preprocessor=ColumnTransformer(
    [
        ("scaler",StandardScaler(),numerical_col),
        ("ohe",OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"),nominal_col),
        ("ordinal",OrdinalEncoder(categories=[traffic_groups,city_groups]),ordinal_col)
    ],remainder='passthrough'
)
preprocessor.set_output(transform="pandas")

ColumnTransformer(remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 ['age', 'ratings', 'vehicle_condition',
                                  'multiple_deliveries', 'is_weekend',
                                  'pickup_time_minutes', 'distance']),
                                ('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'type_of_order', 'type_of_vehicle',
                                  'festival', 'distance_bins',
                                  'order_time_bins']),
                                ('ordinal',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam'],
                                                            ['semi-urban',
                                                             'urban',
                                                             'metropolitian']]),
                                 ['traffic', 'city_type'])])

In [32]:
base_model=RandomForestRegressor()

In [33]:
pipeline1=Pipeline([
    ("preprocessor",preprocessor),
    ("base_model",base_model)
])

In [34]:
reg=TransformedTargetRegressor(
    regressor=pipeline1,
    transformer=PowerTransformer()
)

In [35]:
reg.fit(X_train,Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(remainder='passthrough',
                                                                        transformers=[('scaler',
                                                                                       StandardScaler(),
                                                                                       ['age',
                                                                                        'ratings',
                                                                                        'vehicle_condition',
                                                                                        'multiple_deliveries',
                                                                                        'is_weekend',
                                                                                        'pickup_time_minutes',
                                                                                        'distance']),
                                                                                      ('ohe',
                                                                                       OneHotEncoder(drop='first',
                                                                                                     handle_unknown='ignore',
                                                                                                     sparse_output=False),
                                                                                       ['weather',
                                                                                        'type_of_order',
                                                                                        'type_of_vehicle',
                                                                                        'festival',
                                                                                        'distance_bins',
                                                                                        'order_time_bins']),
                                                                                      ('ordinal',
                                                                                       OrdinalEncoder(categories=[['low',
                                                                                                                   'medium',
                                                                                                                   'high',
                                                                                                                   'jam'],
                                                                                                                  ['semi-urban',
                                                                                                                   'urban',
                                                                                                                   'metropolitian']]),
                                                                                       ['traffic',
                                                                                        'city_type'])])),
                                                     ('base_model',
                                                      RandomForestRegressor())]),
                           transformer=PowerTransformer())

In [36]:
y_pred_test=reg.predict(X_test)
y_pred_train=reg.predict(X_train)

In [37]:
print(f"The train mean absolute error is:{round(mean_absolute_error(Y_train,y_pred_train),2)} min")
print(f"The test mean absolute error is:{round(mean_absolute_error(Y_test,y_pred_test),2)} min")

The train mean absolute error is:1.15 min
The test mean absolute error is:3.14 min


In [38]:
print(f"The train r2 score is:{round(r2_score(Y_train,y_pred_train),2)}")
print(f"The test r2 score is:{round(r2_score(Y_test,y_pred_test),2)}")

The train r2 score is:0.98
The test r2 score is:0.82


In [39]:
pipeline2=Pipeline([
    ("preprocessor",preprocessor),
])

In [40]:
X_train_trans=pipeline2.fit_transform(X_train)
X_test_trans=pipeline2.transform(X_test)

In [41]:
X_train_trans

,scaler__age,scaler__ratings,scaler__vehicle_condition,scaler__multiple_deliveries,scaler__is_weekend,scaler__pickup_time_minutes,scaler__distance,ohe__weather_fog,ohe__weather_sandstorms,ohe__weather_stormy,...,ohe__festival_yes,ohe__distance_bins_medium,ohe__distance_bins_short,ohe__distance_bins_very long,ohe__order_time_bins_afternoon,ohe__order_time_bins_evening,ohe__order_time_bins_morning,ohe__order_time_bins_night,ordinal__traffic,ordinal__city_type
42620,-1.319586,0.528127,0.002935,0.437621,-0.615678,0.006393,0.198212,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
6159,0.936057,-0.104873,-1.221503,0.437621,-0.615678,1.230791,-1.468722,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
18429,0.415524,0.844627,1.227374,-1.310051,1.624227,0.006393,-0.933730,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
2206,-1.319586,0.211627,1.227374,0.437621,-0.615678,0.006393,-1.199370,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
11565,0.762546,-0.737873,0.002935,-1.310051,-0.615678,1.230791,-0.434219,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12203,-0.278520,0.211627,1.227374,0.437621,1.624227,0.006393,-0.633418,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
11198,-0.452031,1.161127,0.002935,0.437621,-0.615678,1.230791,-0.078964,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
33459,1.109568,-2.636872,1.227374,2.185294,1.624227,-1.218005,1.861649,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0
34862,-0.625542,0.844627,0.002935,-1.310051,1.624227,-1.218005,-1.210381,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


#### Random Forest Hyperparameter tuning:

In [44]:
def objective(trial):
  with mlflow.start_run():
    params = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
      'max_depth': trial.suggest_int('max_depth', 3, 30),
      'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
      'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
      'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
      'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    mlflow.log_params(params)

    rf=RandomForestRegressor(**params)

    model=TransformedTargetRegressor(
        regressor=rf,
        transformer=PowerTransformer("yeo-johnson")
    )
    model.fit(X_train_trans,Y_train)

    y_pred_train=model.predict(X_train_trans)
    y_pred_test=model.predict(X_test_trans)

    error=mean_absolute_error(Y_test,y_pred_test)

    scores=cross_val_score(model,X_train_trans,Y_train,cv=3,n_jobs=-1).mean()

    mlflow.log_metric("cross_val_score",-scores)

    return error

In [46]:
study=optuna.create_study(direction="minimize",study_name="best_model_rf")
with mlflow.start_run(run_name="best_model_rf"):
    study.optimize(objective,n_trials=30,n_jobs=-1)
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_score",study.best_value)

    rf=RandomForestRegressor(**study.best_params)
    model=TransformedTargetRegressor(
        regressor=rf,
        transformer=PowerTransformer("yeo-johnson")
    )

    model.fit(X_train_trans,Y_train)

    y_pred_train=model.predict(X_train_trans)
    y_pred_test=model.predict(X_test_trans)

    scores=cross_val_score(model,X_train_trans,Y_train,cv=3,n_jobs=-1).mean()

    #Log metrics:
    mlflow.log_metric("training_error",mean_absolute_error(Y_train,y_pred_train))
    mlflow.log_metric("testing_error",mean_absolute_error(Y_test,y_pred_test))
    mlflow.log_metric("training_r2",r2_score(Y_train,y_pred_train))
    mlflow.log_metric("testing_r2",r2_score(Y_test,y_pred_test))
    mlflow.log_metric("cross_val_score",-scores.mean())

    #Log the best model:
    mlflow.sklearn.save_model(model, "rf_model")
    mlflow.log_artifact("rf_model")






[I 2025-09-22 09:12:26,727] A new study created in memory with name: best_model_rf
[I 2025-09-22 09:14:08,240] Trial 0 finished with value: 3.2402610011625725 and parameters: {'n_estimators': 104, 'max_depth': 26, 'min_samples_split': 19, 'min_samples_leaf': 12, 'max_features': None, 'bootstrap': False}. Best is trial 0 with value: 3.2402610011625725.


🏃 View run nervous-shrimp-738 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0/runs/1d73243bb5f14ada89c7e11c650c7bf9
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0


[I 2025-09-22 09:16:34,416] Trial 2 finished with value: 3.1174914561235867 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 19, 'max_features': None, 'bootstrap': True}. Best is trial 2 with value: 3.1174914561235867.


🏃 View run capable-trout-171 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0/runs/d671b94e5bf04f319b8d20a769f20ffd
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0


[I 2025-09-22 09:20:58,236] Trial 3 finished with value: 3.328325204275784 and parameters: {'n_estimators': 583, 'max_depth': 22, 'min_samples_split': 20, 'min_samples_leaf': 20, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 2 with value: 3.1174914561235867.


...

🏃 View run sneaky-snake-520 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0/runs/4117b76c94724a10b6aa5444f35a35a8
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0


[I 2025-09-22 10:06:04,840] Trial 29 finished with value: 3.3180806129477247 and parameters: {'n_estimators': 777, 'max_depth': 17, 'min_samples_split': 17, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 26 with value: 3.085114853795405.


🏃 View run classy-loon-155 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0/runs/66e04f4ae3144938ac88e60bcd35690e
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


🏃 View run best_model_rf at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0/runs/4f305ccade9948e7a789b75702b2c8b4
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/0


In [47]:
study.best_value

3.085114853795405

In [48]:
study.best_params

{'n_estimators': 542,
 'max_depth': 17,
 'min_samples_split': 15,
 'min_samples_leaf': 10,
 'max_features': None,
 'bootstrap': True}